# <a id='toc1_'></a>[Time Series Forecasting Benchmark](#toc0_)

**Table of contents**<a id='toc0_'></a>    
- [Time Series Forecasting Benchmark](#toc1_)    
  - [Data Setup](#toc1_1_)    
    - [Holt-Winter Triple Exponential smoothing forecasting](#toc1_1_1_)    
      - [Pre-requisites: Exponential Smoothing (ES)](#toc1_1_1_1_)    
      - [History of Development](#toc1_1_1_2_)    
        - [Damped Holt's double exponential smoothing](#toc1_1_1_2_1_)    
    - [Gray Forecasting](#toc1_1_2_)    
    - [Fuzzy Logic](#toc1_1_3_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [18]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

#setup system path for modular import
import sys, os
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.append(project_root)

#Processing and tests
from scipy import stats
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import StandardScaler

#Fetching stock data with the given name
import yfinance as yf

#for technical analysis
import ta
import mplfinance as mpf
import plotly.graph_objects as go

#Time series decomposition and forecasting prep
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller, acf, pacf, kpss

#Forecasting models
from statsmodels.tsa.holtwinters import SimpleExpSmoothing, Holt, ExponentialSmoothing # Holt-Winter

#Evaluation
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [4]:
from src.utils.processing.denoising import Dimensional_Denoise
from src.utils.processing.extract_stock_index import extract_index
from src.utils.processing.feature_extraction import Feature

## <a id='toc1_1_'></a>[Data Setup](#toc0_)

In [47]:
df = extract_index('BA') #Take Boeing for sample univariate dataset

df.head()

,Open,High,Low,Close,Volume,Date
0,64.148095,64.801726,64.064291,64.583847,5036400,2013-01-02
1,64.516799,65.346413,63.997250,64.919037,3987000,2013-01-03
2,65.187206,65.379941,64.893911,65.103409,3431700,2013-01-04
3,64.977707,65.103407,63.134126,63.796139,7172600,2013-01-07
4,63.050340,63.527993,61.265424,62.120171,20268100,2013-01-08


In [53]:
## Split training and testing classically

train = df[df['Date'] < '2022-01-01']
test = df[df['Date'] > '2022-01-01'] # Split 8 years train and 2 years test

### AutoRegressive Integrated Moving Average (ARIMA)

#### Pre-requisites: The Moving Average (MA)

### <a id='toc1_1_1_'></a>[Holt-Winter Triple Exponential smoothing forecasting](#toc0_)


#### <a id='toc1_1_1_1_'></a>[Pre-requisites: Exponential Smoothing (ES)](#toc0_)

Unlike Simple Moving Average (SMA), which treats the last n-datapoints evenly with the same weight = 1/n, the Exponential Moving Average (EMA) algorithm is a more refined version that emphasizes the recent data points more heavily, which makes it more sensitive to recent changes in price.

While EMAs respond more quickly to recent data, they can produce false signal (which is the underlying problem of lag algorithms); however, they will be more effective when used alongside other technical indicators

The General formula for EMA is

$$EMA_t = (P_t.\frac{S}{1 + d} + EMA_{t-1}.(1- \frac{S}{1+d}))$$

Where: 

* $P_t$: Today's Price
* S: Smoothing factor, the most common choice will be 2
* d: Number of days in a moving window, shorter-period EMAs give more weight to recent prices than longer-period EMAs

The Exponential Smoothing Technique work similarly to Exponential Moving Average in the behavior of putting large weight to more recent observations: 

$$y_t = \alpha y_{t-1} + \alpha(1-\alpha) y_{t-2} + \alpha(1- \alpha^2) y_{t-3} + .....$$

and as the observations come further to the past - the weights decrease exponentially with $0<\alpha<1$ being the smoothing parameter

Tuning the parameter $\alpha$ gives larger weights to observations in distant past when $\alpha$ is small (close to 0) and gives larger weights to recent observations when $\alpha$ is large (close to 1)

The Holt-Winters forecasting is one of the most popular forecasting techniques for time series, where it's used for purposes such as anomaly detection and capacity planning.

The H-W forecasting model 3 aspects of the time series: 

* A typical value (average) or level of the series
* A slope (trend) over time
* A cyclical repeating pattern (seasonality) for example m = 4 for quarterly or m = 12 for monthly

each of these has a distinct exponential smoothing series with a smoothing factor, as 3 smoothing factors are required for this approach, this is generally called the H-W Triple Exponentil Smoothing Formula  

The H-W method uses exponential smoothing to encode lots of values from the past and use them to predict the "typical" value in the future. The model requires several parameters: one for each smoothing ($\alpha, \beta, \gamma$), the length of the season, and the number of periods in the season

#### <a id='toc1_1_1_2_'></a>[History of Development](#toc0_)

The model started by a Simple Exponential Smoothing technique to forecast the level (average) of the time series and didn't take into account trend or seasonality $$S_t = \alpha Y_t + (1-\alpha) S_{t-1}$$ and the Forecast for the next period (t+1) is the last calculated smoothed statistic $S_t$

$$F_{t+1} = S_t$$

Then Holt (1957) extended this baseline to allow the forecasting of data with a trend. 

Forecast equation: $$\hat{y}_{t+h|t} = \ell_t + hb_t$$

Level equation: $$\ell_t = \alpha y_t + (1-\alpha)(\ell_{t-1} + b_{t-1})$$

Trend equation: $$b_t = \beta (\ell_t - \ell_{t-1}) + (1 - \beta)b_{t-1}$$

where:
* l_t denotes and estimate of the level of the series at time t
* b_t denotes and estimate of the trend (slope) of the series at time t
* $\alpha$ is the smoothing parameter for the level
* $\beta$ is the smoothing factor for the trend
* h is the forecast horizon

$$0< \alpha, \beta < 1$$

This improvement allows the algorithm to capture the linear trend, work better with stable trend; however, for long horizon (large h), this can lead to unrealistic forecasts (too steep upward or downward)

Therefore, to overcome the issue of long-term trend loss, Gardner & McKenzie (1985) introduced a damping parameter $\theta$ that gradually reduces the trend contribution over time

##### <a id='toc1_1_1_2_1_'></a>[Damped Holt's double exponential smoothing](#toc0_)

Level: $$\ell_t = \alpha y_t + (1-\alpha)(\ell_{t-1} + \phi b_{t-1})$$

Trend: $$b_t = \beta(\ell_t - \ell_{t-1}) + (1-\beta)\phi b_{t-1}$$

Forecast: $$\hat{y}_{t+h} = \ell_t + \left( \frac{1-\phi^h}{1-\phi} \right)b_t, \quad 0<\phi<1$$

* The damping parameter $\phi$ shrinks the trend impact as h grows (longer forecast horizon)

* If $\phi = 1$ -> same as Holt's method (no damping)

* If $\phi = 0$ -> reduces to SES (no trend contribution)

* If $\phi$ stays between 0 and 1 -> the forecast flattens out at long horizons, which is more realistic

Winters later on extended this by adding one more seasonality exponential smoothing term with smoothing factor $\gamma$, making the algorithm fit better with the time series 

The Holt-Winters Triple Exponential Smoothing (additive)

* Level: $$\ell_t = \alpha (y_t - s_{t-m}) + (1-\alpha)(\ell_{t-1} + b_{t-1})$$
$y_t - s_{t-m}$: current level value when excluding seasonal pattern, $\ell_{t-1} + b_{t-1}$: the forecast from the previous timestep
* Trend: $$b_t = \beta (\ell_t - \ell_{t-1}) + (1-\beta)b_{t-1}$$
$\ell_t - \ell_{t-1}$: The observed slope in the level

* Seasonality: $$s_t = \gamma (y_t - \ell_{t-1} - b_{t-1}) + (1-\gamma)s_{t-m}$$

* Forecast: $$\hat{y}_{t+h} = \ell_t + h b_t + s_{t+h-m}$$

There's another approach in Holt-Winters, which is multiplicative, replacing "+" with ".". The additive form is appropriate when the seasonal variation is roughly constant in magnitude, regardless of the level (average)


In [ ]:
# --- Simple Exponential Smoothing ---
simple_model = ExponentialSmoothing(train['Close'], trend = 'mul', initialization_method = 'estimated', seasonal = 'mul', seasonal_periods = 7)
simple_fit = simple_model.fit(smoothing_level=0.9, optimized=True)
simple_forecast = simple_fit.forecast(len(test))

# --- Holt-Winters Exponential Smoothing ---
holt_model = ExponentialSmoothing(train, trend="add", seasonal='add', damped_trend=False, seasonal_periods = 12) #when trend is used, we can all Double Exponential Smoothing, 
                                                                                                                # and when seasonal is also trigered, we can call it Triple
holt_fit = holt_model.fit(smoothing_level=0.8, smoothing_trend=0.2) 
holt_forecast = holt_fit.forecast(len(test))

# --- Damped Holt’s Forecasting ---
dholt_model = ExponentialSmoothing(train, trend="add", seasonal=None, damped_trend=True)
dholt_fit = dholt_model.fit(smoothing_level=0.8, smoothing_trend=0.2, damping_trend=0.9)
dholt_forecast = dholt_fit.forecast(len(test))

# --- Holt’s Forecasting (statsmodels Holt) ---
holt2_model = Holt(train, damped_trend=False)
holt2_fit = holt2_model.fit(smoothing_level=0.8, smoothing_trend=0.2, optimized=True)
holt2_forecast = holt2_fit.forecast(len(test))

In [ ]:
# ...existing code...

plt.figure(figsize=(10, 3))
sns.lineplot(x=train['Date'], y=train['Close'], label='Train', color='black')
sns.lineplot(x=test['Date'], y=test['Close'], label='Test', color='orange')
sns.lineplot(x=test['Date'], y=try_bc, label='Simple ES Forecast', color='red', linestyle='--')
plt.title('Train, Test, and Simple ES Forecast')

plt.xlabel('Date')
plt.ylabel('Close')
plt.legend()
plt.show()

# Successfully capture the trend but not level and seasonality, will require improvement in the future.

When forecasting, add an error bands to show confidence level for future forecast by adding residual variance

The Integration of Lasso and HHT in TS Forecasting

After HHT, we get
* A set of Hilbert transformed signals from IMFs
* Instantaneous frequency
* Instantaneous amplitude

which introduces sparsity, which means not all components carry useful predictive information but rather just noise-dominated and redundant

Lasso (L1 penalty) enforces sparsity by shrinking many coefficients exactly to 0 => perfect for HHT, as don't want all IMFs, just relevant ones. so Lasso will filters out noise features 

### <a id='toc1_1_2_'></a>[Gray Forecasting](#toc0_)

### <a id='toc1_1_3_'></a>[Fuzzy Logic](#toc0_)